In [ ]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Json"
  // #r "nuget: Plotly.NET.ImageExport"

open Plotly.NET
// open Plotly.NET.ImageExport  

#load "Helpers.fs"
#load "Bestiary.fs"
#load "Compare.fs"
#load "DamageDistribution.fs"
#load "Library.fs"
#load "Transform.fs"
#load "ChartTools.fs"

open PathfinderAnalysis.Library
open PathfinderAnalysis.Bestiary
open PathfinderAnalysis.Compare
open PathfinderAnalysis.Transform
open PathfinderAnalysis.Helpers
open PathfinderAnalysis.DamageDistribution
open PathfinderAnalysis.ChartTools

In [ ]:
// cantrip tier - benchmark: 9.5
let creatureLevelBump = 2

let titleFn level = 
  sprintf "Middle Save - Level %i - Spout Cantrip vs. Martial Bow Against Level %i Creatures" level (level+creatureLevelBump)

[|
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel creatureLevelBump
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "🤺 Arbalest (1 shot) ❖"
  };
  {
    AveragesByRollsByLevel = mapMerge (highMartialAttack true) creatureLevelBump 5 martialArbalest;
    Title = "🤺 Arbalest (2 shots) ❖❖❖"
  };
  {
    AveragesByRollsByLevel = mapMerge (highMartialAttack true) creatureLevelBump 5 martialShortbow;
    Title = "🤺 Shortbow (2 shots) ❖❖"
  };
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel telekineticProjectile PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel creatureLevelBump
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "🩸 Telekinetic Projectile ❖❖"
  };
  standardSpellChart "🌿 Spout (Middle) ❖❖" spout SaveSelector.Middle creatureLevelBump;
  standardSpellChart "🌿 Spout (Lowest) ❖❖" spout SaveSelector.Lowest creatureLevelBump;
  standardSpellChart "🌿 Spout (Highest) ❖❖" spout SaveSelector.Highest creatureLevelBump;
  standardSpellChart "🌿 Spout (Reflex) ❖❖" spout SaveSelector.Reflex creatureLevelBump;
  // { 
  //   AveragesByRollsByLevel = transformedResultsByRollByLevel tempestSurge CreatureSave highestSave (casterDc true) bestiaryByLevel creatureLevelBump
  //     |> Seq.map resultRollsToAverages
  //     |> Seq.map normalizeSavingThrowsForLevel
  //     |> Seq.toArray;
  //   Title = "🌿 Tempest Surge (Highest) ❖❖"
  // };
|]
|> flatten
// |> generateCharts titleFn
|> generateLevelScaleChart (sprintf "Caster and Martial Backup Options (no Property Runes) - Creatures PL+%i" creatureLevelBump)
// |> Seq.iteri (fun i chart -> Chart.savePNG (path = (sprintf "spout-arbalest-%02i" i)) chart)
// |> ignore


In [ ]:
// focus spell tier - benchmark: 21
let creatureLevelBump = 2

let titleFn level = 
  sprintf "Level %i - Focus Spells and Favored Ranged Weapons Against Level %i Creatures" level (level+creatureLevelBump)

[|
  fighterLongbowDoubleShotChart creatureLevelBump;
  fighterLongbow2StrikesChart creatureLevelBump;
  sorcererDragonBreath1MiddleTargetChart creatureLevelBump;
  clericFireRayMovesChart creatureLevelBump;
  clericFireRayStaysChart creatureLevelBump;
  druidTempestSurgeChart SaveSelector.Middle creatureLevelBump;
  druidTempestSurgeChart SaveSelector.Lowest creatureLevelBump;
  druidTempestSurgeChart SaveSelector.Highest creatureLevelBump;
  druidTempestSurgeChart SaveSelector.Reflex creatureLevelBump;
  // fighterLongbowIncredibleAimChart creatureLevelBump;
  // fighterLongbow3StrikesChart creatureLevelBump;
  // fighterLongbowTripleShotChart creatureLevelBump;
  // fighterLongbowDoubleShotThenStrikeChart creatureLevelBump;
  // fighterLongbowStrikeThenDoubleShotChart creatureLevelBump;
  // fighterLongbowIncredibleAimThenStrikeChart creatureLevelBump;
  // fighterLongbowMultiShotStanceTripleShotChart creatureLevelBump;
  // highestRankForceBarrage2ActionChart creatureLevelBump;
  // highestRankForceBarrage3ActionChart creatureLevelBump;
|]
|> flatten
// |> generateCharts titleFn 
|> generateLevelScaleChart (sprintf "Focus Spells and Favored Ranged Weapons - Creatures PL+%i" creatureLevelBump)


In [ ]:
// slotted spell tier - benchmark: 30
let creatureLevelBump = 2

let titleFn level = 
  sprintf "Level %i - Caster Single-Target Ranged Spells and Martial Melee Weapons - Level %i Creatures" level (level+creatureLevelBump)

[|
  barbarianGreatsword2StrikesChart creatureLevelBump;
  fighterGreatsword2StrikesChart creatureLevelBump;
  fighterDoubleSliceChart creatureLevelBump;
  // highestRankForceBarrage2ActionChart creatureLevelBump;
  highestRankForceBarrage3ActionChart creatureLevelBump;
  druidThunderstrikeChart SaveSelector.Middle creatureLevelBump;
  druidThunderstrikeChart SaveSelector.Lowest creatureLevelBump;
  druidThunderstrikeChart SaveSelector.Highest creatureLevelBump;
  druidThunderstrikeChart SaveSelector.Reflex creatureLevelBump;
  clericFireRayStaysChart creatureLevelBump;
|]
|> flatten
// |> generateCharts titleFn
|> generateLevelScaleChart (sprintf "Caster Single-Target Ranged Spells and Martial Melee Weapons - Creatures PL+%i" creatureLevelBump)

In [ ]:
// damage mapped to d20
open PathfinderAnalysis.DamageDistribution

let dicePool = [D12, 4; D6, 3; D4, 2]
let nBuckets = 20

let values = 
  rollDistributions 0 dicePool
  |> Seq.toList
  |> chunkRolls nBuckets
  |> chunksToAverages
  |> Seq.toList

let title = sprintf "Roll Sum of %A mapped onto a d%i" dicePool nBuckets

Chart.Line(
  x = List.map first values,
  y = List.map second values,
  ShowMarkers = true,
  MultiText = (List.map second values |> List.map (sprintf "%.0f")),
  TextPosition = StyleParam.TextPosition.TopCenter
)
|> Chart.withXAxisStyle(TitleText = "D20 Result")
|> Chart.withYAxisStyle(TitleText = "Estimated Roll Sum")
|> Chart.withTitle(title)



In [ ]:
// turn luck mapped to d20
#load "ChartTools.fs"
open PathfinderAnalysis.ChartTools

open PathfinderAnalysis.DamageDistribution

let creatureLevelBump = 2

let titleFn level =
  sprintf "Level %i - Druid vs. Fighter Against Level %i Creatures (AC: %.1f DC: %.1f)" level (level + creatureLevelBump) (average creatureAc level) (average middleSave level + 10.0)

let fighterLongbowBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongbow pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongbow pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

// let fighterShortbowBucketsForLevel pcLevel =
//   [|
//     { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterShortbow pcLevel };
//     { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterShortbow pcLevel };
//   |]
//   |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let druidReflexBuckets pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = (fun c -> c.reflex + c.hasMagicBonus); DamageFunction = diceDamageTempestSurge pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let druidMiddleBuckets pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = middleSave; DamageFunction = diceDamageTempestSurge pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let druidMiddleThunderstrikeBuckets pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = middleSave; DamageFunction = diceDamageThunderstrike pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let sorcererMiddleThunderstrikeBuckets pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = middleSave; DamageFunction = diceSorcererThunderstrike pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let fighterGreatswordBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterGreatsword pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterGreatsword pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let fighterLongswordBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongsword pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongsword pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let barbarianGreatswordBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highMartialAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceBarbarianGreatsword pcLevel };
    { HitRollCount = 1; HitModifier = highMartialAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceBarbarianGreatsword pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let fighterDoubleSliceBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongsword pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterShortsword pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

// [1;5;7;13;15]
[13;15]
|> List.map (fun level -> [
  // "Tempest Surge (Reflex)", level, druidReflexBuckets level;
  // "Longbow (2 Strikes)", level, fighterLongbowBucketsForLevel level;
  // "Tempest Surge (Middle)", level, druidMiddleBuckets level;
  // "Shortbow (2 Strikes)", level, fighterShortbowBucketsForLevel level
  // "⚔️ Longsword (2 Strikes) ❖❖", level, fighterLongswordBucketsForLevel level;
  // "⚔️ Greatsword (2 Strikes) ❖❖", level, fighterGreatswordBucketsForLevel level;
  "🐉 Greatsword (2 Strikes) ❖❖", level, barbarianGreatswordBucketsForLevel level;
  // "🌿 Thunderstrike (Middle) ❖❖", level, druidMiddleThunderstrikeBuckets level;
  "🩸 Thunderstrike (Middle) ❖❖", level, sorcererMiddleThunderstrikeBuckets level;
  "⚔️ Double Slice (Swords) ❖❖", level, fighterDoubleSliceBucketsForLevel level;
])
|> Seq.collect (fun titledBucketsByLevel -> Seq.map (fun (title, level, bucket) -> chunksToFlatChartData title level bucket) titledBucketsByLevel)
// |> generateChartsPercent titleFn creatureLevelBump
|> generateChartsTTK titleFn creatureLevelBump
// |> generateTTKHistogram titleFn creatureLevelBump
// |> generateLevelScaleChart (sprintf "Reflex Save - Druid vs. Fighter Creatures PL+%i" creatureLevelBump)


In [ ]:
// bigthing histogram

#load "ChartTools.fs"
open PathfinderAnalysis.ChartTools

let creatureLevelBump = 2

let titleFn level =
  sprintf "Level %i - Turn Damage vs. Level %i Creatures (AC: %.1f DC: %.1f)" level (level + creatureLevelBump) (average creatureAc level) (average middleSave level + 10.0)

let fighterDoubleSliceAll pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongsword pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterShortsword pcLevel };
  |]
  |> theBigThing bestiaryByLevel[pcLevel + creatureLevelBump]

let sorcererMiddleThunderstrikeAll pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = middleSave; DamageFunction = diceSorcererThunderstrike pcLevel };
  |]
  |> theBigThing bestiaryByLevel[pcLevel + creatureLevelBump]

let barbarianGreatswordTwiceAll pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highMartialAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceBarbarianGreatsword pcLevel };
    { HitRollCount = 1; HitModifier = highMartialAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceBarbarianGreatsword pcLevel };
  |]
  |> theBigThing bestiaryByLevel[pcLevel + creatureLevelBump]

let wizardForceBarrageAll actions pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterAttack true false pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceDamageForceBarrage actions pcLevel  }
  |]
  |> theBigThing bestiaryByLevel[pcLevel + creatureLevelBump]
[9]
|> Seq.map (fun level -> [
  "🩸 Thunderstrike (Middle) ❖❖", level, sorcererMiddleThunderstrikeAll level;
  "⚔️ Double Slice (Swords) ❖❖", level, fighterDoubleSliceAll level;
  "🐉 Greatsword (2 Strikes) ❖❖", level, barbarianGreatswordTwiceAll level;
  "🧙 Force Barrage ❖❖", level, wizardForceBarrageAll 2 level;
  "🧙 Force Barrage ❖❖❖", level, wizardForceBarrageAll 3 level;
])
|> generateShape titleFn
  